In [ ]:
## Bird specious Classification ##
""" Our constructed dataset structure is as follow:
   src_dir:
        Familly 1 
                  Class 1_1
                  Class 1_2
                     .
                     .
                     .
        
        Familly 2 
                  Class 2_1
                  Class 2_2
                      .
                      .
            .
            .
        Familly n
                  Class n_1
                  Class n_2
                       .
                       .
        
"""


In [5]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from keras.models import Model, sequential
from keras.layers import Input, Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from keras.utils import layer_utils
from keras import backend as K
from keras.optimizers import RMSprop, SGD, Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard, CSVLogger
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.applications import xception
from keras.applications import inception_v3
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator


In [4]:
############  Load Dataset ############

dataset_url = 'https://entuedu-my.sharepoint.com/personal/copp0001_e_ntu_edu_sg/_layouts/15/onedrive.aspx?e=5%3a0ac34a4383a14ce6aaae574c0712f669&at=9&CT=1540948068080&OR=OWA-NT&CID=956c4b9f-6ba1-4f5e-b424-9e908f094666&id=%2fpersonal%2fcopp0001_e_ntu_edu_sg%2fDocuments%2fPhD%2fCourses%2fDL%2fbirds_of_singapore&FolderCTID=0x012000E09AF0217322FC4FAD0767E54F1F0F07/dataset_complete.csv'
data = pd.read_csv(dataset_url)

y1 = data.temp(y=data.temp(axis=2))
y2 = data.temp(y=data.temp(axis=4))
X = data.drop('temp', axis=1)

HTTPError: HTTP Error 403: Forbidden

In [ ]:
############ Split data for two models  ############
n_examples = 21889 #number of dataset samples

X1_train, X1_test, Y1_train, Y1_test = train_test_split(X, y1, test_size=0.3, random_state=42)
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X, y2, test_size=0.3, random_state=42)
#print(X1_train.shape)
#print(X1_test.shape)

In [ ]:
X1_train = np.asarray(X1_train).reshape((-1, 250, 250, 3))
X1_test = np.asarray(X1_test).reshape((-1, 250, 250, 3))
X2_train = np.asarray(X2_train).reshape((-1, 250, 250, 3))
X2_test = np.asarray(X2_test).reshape((-1, 250, 250, 3))

Y1_train = np_utils.to_categorical(np.asarray(Y1_train), nb_classes=44)
Y1_test = np_utils.to_categorical(np.asarray(Y1_test), nb_classes=44)

Y2_train = np_utils.to_categorical(np.asarray(Y2_train), nb_classes=96)
Y2_test = np_utils.to_categorical(np.asarray(Y2_test), nb_classes=96)

print X1_train.shape, X1_test.shape
print Y1_train.shape, Y1_test.shape

split1 = int(0.8 * X1_train.shape[0])
split2 = int(0.8 * X2_train.shape[0])

x1_val = X1_train[split1:]
x1_train = X1_train[:split1]
y1_val = Y1_train[split1:]
y1_train = Y1_train[:split1]

x2_val = X2_train[split2:]
x2_train = X2_train[:split2]
y2_val = Y2_train[split2:]
y2_train = Y2_train[:split2]
### Save 
#dir = '/Users/mitra/
x1_train.to_csv('dataset/training/X1_train.csv',sep=",")
x1_test.to_csv('dataset/validation/X1_test.csv',sep=",")
x2_train.to_csv('dataset/training/X2_train.csv',sep=",")
x2_test.to_csv('dataset/validation/X2_test.csv',sep=",")

In [ ]:
############ Import ResNet50 weights trained on ImageNet ##########
model = ResNet50(weights='imagenet')
model = ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

In [ ]:
############ Add 3 hidden layers and an output layer ############
x = model.output
x = Flatten()(x)
x=Dropout(0.25)(x)
x=Dense(units=150,activation='relu')(x)
x=Dropout(0.25)(x)
x=Dense(units=750,activation='relu')(x)
x=Dropout(0.25)(x)
x=Dense(units=750,activation='relu')(x)
x=Dropout(0.25)(x)
x=Dense(units=750,activation='relu')(x)
x=Dropout(0.25)(x)
x=Dense(units=150,activation='relu')(x)
x=Dropout(0.6)(x)
#clf.add(Dense(units=120,activation='softmax')
#stochastic gradient descent -Adam -optimizer
#loss func categorical cross entropy
#metrics = accuracy
#clf.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
predictions = Dense(120, activation='softmax')(x)

In [ ]:
main_model = Model(inputs=model.input, outputs=predictions)
#train the hidden layers and output layer
for curLayer in model.layers:
    curLayer.trainable = False

main_model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
main_model.summary()

In [ ]:
############  data Augmentation ############ 
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)
training_set1 = train_datagen.flow_from_directory(
        '/training/X1_train.csv',
        target_size=(250, 250),
        batch_size=20,
        class_mode='categorical')
training_set2 = train_datagen.flow_from_directory(
        '/training/X2_train.csv',
        target_size=(250, 250),
        batch_size=20,
        class_mode='categorical')

test_set1 = test_datagen.flow_from_directory(
        '/validation/X1_test.csv',
        target_size=(250, 250),
        batch_size=22,
        class_mode='categorical')
test_set2 = test_datagen.flow_from_directory(
        '/validation/X2_test.csv',
        target_size=(250, 250),
        batch_size=22,
        class_mode='categorical')
""""
hist=main_model.fit_generator(
        training_set,
        steps_per_epoch=400,
        epochs=25,
        validation_data=test_set,
        validation_steps=101,callbacks=callbacks_list)
""""

In [ ]:
############  Model fitting ############ 
hist1 = model.fit(training_set1, y1_train,
         nb_epoch=20,
         shuffle=True,
         batch_size=100,
         validation_data=(test_set1, y1_val), verbose=2)

or l in model.layers[:6]:
    l.trainable = False   

# In[ ]:

trans_model = Sequential(model.layers[:6])

trans_model.add(Dense(128))
trans_model.add(Activation('relu'))
trans_model.add(Dense(10))
trans_model.add(Activation('softmax'))

trans_model.summary()
trans_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
hist2 = trans_model.fit(training_set2, y2_train, nb_epoch=10, shuffle=True, batch_size=100, validation_data=(test_set2, y2_val), verbose=2)

In [ ]:
""""
classes= {index:specious for specious,index in training_set.class_indices.items()}
column_names = [classes[i] for i in range()]


predictions_df = pd.DataFrame(predictions)
predictions_df.columns = column_names
predictions_df.insert(0,'id', test_set_ids)
predictions_df.set_index('id',inplace=True)
predictions_df
""""

In [ ]:
#### save
predictions_df.to_csv('project/output/final_class.csv',sep=",")